<a href="https://colab.research.google.com/github/PerfectMach1ne/learning-google-colab/blob/main/fcc_MAGIC_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on https://www.youtube.com/watch?v=i_LwzRVP7bg and a little bit of https://bebi103a.github.io/lessons/00/setting_up_your_computer.html#Setting-up-Google-Colab (basically everything in this repo is going to be done based off of those 2 sources.)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
cols = ["fLength", "fWidth", "fSize", "fConc", "fConc1", "fAsym", "fM3Long", "fM3Trans", "fAlpha", "fDist", "class"]
df = pd.read_csv('magic04.data', names=cols) # df = dataframe
df.head() # "gimme the first 5 things"

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [13]:
df["class"].unique()

array(['g', 'h'], dtype=object)

'g' stands for gammas and 'h' for hadrons.

In [14]:
df["class"] = (df["class"] == 'g').astype(int) # Since we have binary data, we can convert class data representing either
# gammas or hadrons to True and False and then subsequently convert this boolean data to 1s and 0s.